# Capacity and quantum geometry of parametrized quantum circuits


"Capacity and quantum geometry of parametrized quantum circuits" by T. Haug, K. Bharti, M.S. Kim
arXiv:2102.01659

Example code to calculate quantum Fisher information matrix,effective quantum dimension and quantum natural gradient.

Calculates properties of a parametrized quantum circuit 
U(\theta)=W_L R_L(\theta) W_{L-1} R_{L-1}(\theta) ... W_1 R_1(\theta) \sqrt{H}\ket{0}

W_l is entangling layer of two-qubit entangling operations, R_l are single-qubit rotations

Based on qutip

@author: Tobias Haug, github txhaug
Imperial College London
Contact at tobiasxhaug@gmail.com


In [10]:
import qutip as qt

#functools - for 'higher-order functions', creatin functions that operate on functions
from functools import partial 

import operator
from functools import reduce
import numpy as np

import scipy

In [11]:
def prod(factors):
    #applies operator.mul from left to right on the list factors, i.e reduce(mul, [2,3,4,5]) = (((2*3)*4)*5)
    return reduce(operator.mul, factors, 1)


def flatten(l):
    #flattens a 2d list into a 1d list - looks real weird though
    return [item for sublist in l for item in sublist]

#tensors operators together 
def genFockOp(op,position,size,levels=2,opdim=0): #what is opdim - ask Tobias
    #qeye is identity operation - makes 'levels' dimensional Hilbert Space (H)
    opList=[qt.qeye(levels) for x in range(size-opdim)] #size = how many spins/qubits
    opList[position]=op #set positionth H to the given operation
    return qt.tensor(opList) #tensor product these identities and the operation - why is this a Fock op?

In [12]:
test_list = [[1,2,3],[4,5,6],[[7],[8,9]]]
print(flatten(test_list))
qt.qeye(3) #kinda just a n*n identity but has other properties - s a qutip.qobj.Qobj

[1, 2, 3, 4, 5, 6, [7], [8, 9]]


Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Set parameters here for circuit

In [13]:

n_qubits=4 #number qubits
depth=3 #number of layers

#type of entangling gate used
type_entanglers=0 #0: CNOT, 1:CPHASE, 2: \sqrt{iSWAP}

#how to arrange the entangling layer. Fig 1 in paper
entangling_arrangement=0 #0: one-dimensional nearest-neighbor CHAIN, 1: ALl-to-ALL connections
#layer means applying rotation & entanglement sequentially i.e applying W_l p times to the output of previous W_l

In [14]:
#random generator used
rng = np.random.default_rng(1)


#define angles for circuit - a 2D array w/ dimensions (depth, n_qubits) this is theta
# need depth * n qubit params to vary
ini_angles=rng.random([depth,n_qubits])*2*np.pi

#define rotations for circuit in each layer, 1: X, 2:Y 3:Z
ini_pauli=rng.integers(1,4,[depth,n_qubits])


##test case for n_qubits=4, depth=3, type_entanglers=0,entangling_arrangement=0
#ini_angles=np.reshape([1.48304, 2.17723, 1.9648, 0.0496955, 3.07004, 1.32555, 5.98107, 6.28259, 1.58124, 6.19941, 3.49189, 2.74643],[depth,n_qubits])
#ini_pauli=np.reshape([3, 2, 2, 2, 3, 2, 3, 1, 3, 2, 2, 1],[depth,n_qubits])
##should give energy=-0.290501, gradient_mean= 0.112244, qng_mean=0.3352

#depth = # of layers
n_parameters=depth*n_qubits #number of parameters of circuit


cutoff_eigvals=10**-12 #define all eigenvalues of quantum fisher information metric as 0

In [15]:
#operators for circuit, arrays of all diff single qubit operators, done for ease of use
#can just call opZ[n] to operate sigma_z on nth qubit
levels=2
opZ=[genFockOp(qt.sigmaz(),i,n_qubits,levels) for i in range(n_qubits)]
opX=[genFockOp(qt.sigmax(),i,n_qubits,levels) for i in range(n_qubits)]
opY=[genFockOp(qt.sigmay(),i,n_qubits,levels) for i in range(n_qubits)]
opId=genFockOp(qt.qeye(levels),0,n_qubits)


In [16]:
H=opZ[0]*opZ[1] #local Hamiltonian to calculate energy and gradient from
# is sigmaZ on qubit 1 * sigmaZ on qubit 2
entangling_arrangement = 1 #DELETE THIS LATER
#defining structure of circuit and types of entanglers in these two elifs
#define entangling gate arrangement, 
#output is list of connected qubits in each topology, i.e in CHAIN for 4 qubits (0,1), (2,3) and (1,2) are connected per fig 1c 
if(entangling_arrangement==0):    #here is for chain topology
    entangling_gate_index=[[2*j,2*j+1] for j in range(n_qubits//2)]+[[2*j+1,2*j+2] for j in range((n_qubits-1)//2)]
elif(entangling_arrangement==1):##all-to-all
    #randomize control and target for more entangling power for CNOT
    entangling_gate_index=flatten([[rng.permutation([i,j]) for j in range(i+1,n_qubits)] for i in range(n_qubits-1)])
entangling_gate_index
#order of EGI is important, they are listed as applied from L->R

[array([0, 1]),
 array([2, 0]),
 array([3, 0]),
 array([2, 1]),
 array([1, 3]),
 array([2, 3])]

In [17]:
#type of entangliers used
#literally produces an array of gates connecting the index pair in entangling gate index
#need N_qubits so correct size matrix output by qt.qip.operations.gate
if(type_entanglers==0):#CNOT
    entangling_layer=prod([qt.qip.operations.cnot(n_qubits,j,k) for j,k in entangling_gate_index][::-1])#need [::-1] to invert order so that unitaries are multiplied in correct order
elif(type_entanglers==1):#CPHASE
    entangling_layer=prod([qt.qip.operations.csign(n_qubits,j,k) for j,k in entangling_gate_index][::-1])
elif(type_entanglers==2):#\sqrt{iSWAP}
    entangling_layer=prod([qt.qip.operations.sqrtiswap(n_qubits,[j,k]) for j,k in entangling_gate_index][::-1])



In [18]:
#calculate state and gradients

#list of values of gradient
gradient_list=np.zeros(n_parameters)

#save here quantum state of gradient
grad_state_list=[]

#initial state is updated over time, is a bit of a misnomer.


#p goes from -1 to n_parameters-1. -1 is to calculate quantum state, 0 to n_parameters rest for gradient
#calculates the quantum state U\ket{0}, as well as \partial_i U \ket{0}
#those can be used to calcute energy, gradients and the Quantum Fisher metric 
for p in range(-1,n_parameters): #generates gradient of pth parameter - need this to optimise each paramater
    counter=0 #counter to keep track of which parameter is calculated
    initial_state=qt.tensor([qt.basis(levels,0) for i in range(n_qubits)])
    #initial layer of fixed \sqrt{H} rotations
    initial_state=qt.tensor([qt.qip.operations.ry(np.pi/4) for i in range(n_qubits)])*initial_state
    
    #go through depth layers
    for j in range(depth):
        rot_op=[] #rot_op reset every time
        #define parametrized single-qubit rotations at layer j
        for k in range(n_qubits):
            angle=ini_angles[j][k]
            if(ini_pauli[j][k]==1):
                rot_op.append(qt.qip.operations.rx(angle))
            elif(ini_pauli[j][k]==2):
                rot_op.append(qt.qip.operations.ry(angle))
            elif(ini_pauli[j][k]==3):
                rot_op.append(qt.qip.operations.rz(angle))
                
                
            #multiply in derivative of parametrized single-qubit rotation gate at layer j for parameter of circuit p
            #this is the exact derivative
            if(counter==p): #if j,k is parameter you want to measure gradient of then change its initial state like this
                if(ini_pauli[j][k]==1): #c.f eq (16) of nist review - derivative of rotation matrix w.r.t parameter p 
                    initial_state=(-1j*opX[k]/2)*initial_state
                elif(ini_pauli[j][k]==2):
                    initial_state=(-1j*opY[k]/2)*initial_state
                elif(ini_pauli[j][k]==3):
                    initial_state=(-1j*opZ[k]/2)*initial_state
                
            counter+=1 #this goes from 0-12 bc double nested j, k loop
                
        #multiply in single-qbuit rotations 
        #rot_op is each qubit rotation, then apply this to the layer, add entanglement layer and move to next layer
        initial_state=qt.tensor(rot_op)*initial_state
    
        #add entangling layer, applies above tolpology to the state
        initial_state=entangling_layer*initial_state
     
    if(p==-1):#get quantum state for cost function
        #cost function given by <\psi|H|\psi>
        circuit_state=qt.Qobj(initial_state)#state generated by circuit
        energy=qt.expect(H,circuit_state)
        print("Energy of state",energy)

    else:#get quantum state needed for gradient
        grad_state_list.append(qt.Qobj(initial_state))#state with gradient applied for p-th parameter

        #gradient of circuit is given by 2*real(<\psi|H|\partial_p\psi>)
        gradient_list[p]=2*np.real(circuit_state.overlap(H*initial_state))

Energy of state -0.008093523151450746


In [19]:
#quantum fisher information metric
#calculated as \text{Re}(\braket{\partial_i \psi}{\partial_j \psi}-\braket{\partial_i \psi}{\psi}\braket{\psi}{\partial_j \psi})

#first, calculate elements \braket{\psi}{\partial_j \psi})
single_qfi_elements=np.zeros(n_parameters,dtype=np.complex128)
for p in range(n_parameters):
    #print(circuit_state.overlap(grad_state_list[p]))
    single_qfi_elements[p]=circuit_state.overlap(grad_state_list[p]) #overlap intergral of initial state and its resultant gradient <psi|d_j psi>
            

#calculcate the qfi matrix. QFI symmetric
qfi_matrix=np.zeros([n_parameters,n_parameters]) #square matrix
for p in range(n_parameters):
    for q in range(p,n_parameters):
        qfi_matrix[p,q]=np.real(grad_state_list[p].overlap(grad_state_list[q])-np.conjugate(single_qfi_elements[p])*single_qfi_elements[q]) #this is (B3)
    
    
#use fact that qfi matrix is real and hermitian
for p in range(n_parameters):
    for q in range(p+1,n_parameters):  
        qfi_matrix[q,p]=qfi_matrix[p,q]
qfi_matrix #12 params * 4 qubits * 3 layers

array([[ 1.25000000e-01,  9.71445147e-17, -6.05064588e-19,
         1.11022302e-16,  8.81446342e-02,  8.35963762e-02,
         4.16333634e-17,  4.50370592e-04, -7.01457109e-02,
        -7.67105862e-04, -8.01912044e-05,  8.84314696e-02],
       [ 9.71445147e-17,  1.25000000e-01, -1.96870228e-17,
         1.11022302e-16, -8.81446342e-02, -5.55111512e-17,
         8.32208262e-02,  4.50370592e-04, -8.64401376e-02,
        -7.67105862e-04,  1.10778414e-02,  7.43773970e-02],
       [-6.05064588e-19, -1.96870228e-17,  2.50000000e-01,
        -2.42110323e-18, -5.55005844e-18, -1.28739069e-17,
        -3.30644795e-17,  3.07388446e-18,  1.29515101e-17,
         6.97027721e-18, -6.56756559e-02, -1.33483643e-17],
       [ 1.11022302e-16,  1.11022302e-16, -2.42110323e-18,
         1.25000000e-01, -8.32667268e-17,  8.35963762e-02,
        -8.32208262e-02,  4.50370592e-04, -6.38957022e-02,
         1.00541279e-01, -8.01912044e-05, -1.65988358e-02],
       [ 8.81446342e-02, -8.81446342e-02, -5.5500584

Get eigenvalues and eigenvectors of quantum fisher information metric. Gives information about the parameter space of the circuit. Zero eigenvalues indicate redundant parameters, i.e. which do not contribute The larger the eigenvalue, the more the quantum state changes when changing the parameters in the direction of the eigenvector-
For random initial parameters, the effective quantum dimension is equivalent to the parameter dimension, which is the total number of independent parameters that the quantum state generated by the quantum circuit can represent.

In [46]:
#get eigenvalues and eigenvectors of QFI
eigvals,eigvecs=scipy.linalg.eigh(qfi_matrix)

In [47]:
#non-zero eigenvalues of qfi
nonzero_eigvals=eigvals[eigvals>cutoff_eigvals]

#effective quantum dimension, i.e. number of independent directions when perturbing system
#is equivalent to parameter dimension (number of independent parameters of quantum state that can be represented by circuit) for random initial values of circuit
eff_quant_dim=len(nonzero_eigvals)

n_zero_eigval=n_parameters-eff_quant_dim

print("Hilbert space", 2**n_qubits)
print("Number parameters of circuit",n_parameters)
print("Effective quantum dimension G_C",eff_quant_dim)


Hilbert space 16
Number parameters of circuit 12
Effective quantum dimension G_C 12


Number of parameters of circuit that are redundant

In [48]:

#fraction of zero eigenvalues
redundancy=n_zero_eigval/n_parameters
print("redundancy",redundancy)

redundancy 0.0


Logarithm of eigenvalues has peak when the parameter dimension become maximal. Increase depth of circuit to observe this effect.

In [49]:
qfi_var_log_eigval=np.var(np.log10(nonzero_eigvals))

print("Logarithm of non-zero qfi eigenvalues",qfi_var_log_eigval)

Logarithm of non-zero qfi eigenvalues 0.27731686815961337


Gradient and natural gradient. Calculates variance, which decreases when increasing depth of circuit and number of qubits, which is hallmark of barren plateau (or vanishing gradient) problem. Note that variance of quantum natural gradient is larger than regular gradient, but both suffer from barren plateaus. 
When observing peak in logarithm of eigenvalues, the variance of the quantum natural gradient will also decrease also

In [50]:
eigvals_inv=np.zeros(n_parameters)
#invert eigenvalues if they are above threshold, else set to zero
for i in range(n_parameters):
    if(eigvals[i]<cutoff_eigvals):
        eigvals_inv[i]=0 #inverted eigenvalues with cutoff of smallest eigenvalues set to zero
    else:
        eigvals_inv[i]=1/eigvals[i]
        
        

#inverse for quantum natural gradient
#reconstruct inverse matrix from inverse eigenvalues and eigenvectors 
qfi_inv_matrix=np.dot(eigvecs,np.dot(np.diag(eigvals_inv),np.transpose(np.conjugate(eigvecs))))

##qfi_inv_matrix=scipy.linalg.pinv(qfi_matrix)
#quantum natural gradient, is gradient with quantum geometric information applied, moves efficient in parameter space

quantum_natural_gradient=np.dot(qfi_inv_matrix,gradient_list)


mean_gradient=np.mean(gradient_list)
mean_qng=np.mean(quantum_natural_gradient)
variance_gradient=np.var(gradient_list)
variance_qng=np.var(quantum_natural_gradient)

#mean of gradients and qng
print("mean gradient",mean_gradient,"mean qng",mean_qng)
#variance of gradients and qng
print("variance gradient",variance_gradient,"variance qng",variance_qng)

mean gradient 0.017470971850529638 mean qng 0.40691992542668515
variance gradient 0.009358867375641263 variance qng 2.127928944566042
